# Datasets and Parallelization with Spark


## Feng Li

### Central University of Finance and Economics

### [feng.li@cufe.edu.cn](feng.li@cufe.edu.cn)
### Course home page: [https://feng.li/distcomp](https://feng.li/distcomp)

## Two available APIs in Spark

- RDDs (core but old API), accumulators, and broadcast variables
- Spark SQL, Datasets, and DataFrames: processing structured data with relational queries (newer API than RDDs)

## Resilient distributed dataset RDD

- At a high level, every Spark application consists of 

    - a driver program that runs the user’s main function and 
    - executes various parallel operations on a cluster. 

- The main abstraction Spark provides is a **resilient distributed dataset** (RDD), 

    - a collection of elements partitioned across the nodes of the cluster that can be operated on in parallel. 
    - RDDs are created by starting with a file in the Hadoop file system and transforming it. 
    - Users may also ask Spark to persist an RDD in memory, allowing it to be reused efficiently across parallel operations. 
    - Finally, RDDs automatically recover from node failures.


- A second abstraction in Spark is **shared variables** that can be used in parallel operations. 

    - By default, when Spark runs a function in parallel as a set of tasks on different nodes, it ships a copy of each variable used in the function to each task. 
    - Sometimes, a variable needs to be shared across tasks, or between tasks and the driver program. 
    - Spark supports two types of shared variables: 
    
        - **broadcast variables**, which can be used to cache a value in memory on all nodes, 
        - **accumulators**, which are variables that are only “added” to, such as counters and sums.

## Create RDDs

- Spark revolves around the concept of a resilient distributed dataset (RDD), which is a fault-tolerant collection of elements that can be operated on in parallel. 

- There are two ways to create RDDs: 

    - parallelizing an existing collection in your driver program, 
    - referencing a dataset in an external storage system, such as a shared filesystem, HDFS, HBase, or any data source offering a Hadoop InputFormat.



## Create a SparkContext object

- The first thing a Spark program must do is to create a `SparkContext` object, which tells Spark how to access a cluster. To create a `SparkContext` you first need to build a `SparkConf` object that contains information about your application.

In [3]:
import findspark
findspark.init("/usr/lib/spark-current")
import pyspark

In [4]:
conf = pyspark.SparkConf().setAppName("My First Spark RDD APP").setMaster("local")  # “yarn”
sc = pyspark.SparkContext(conf=conf)

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:39092)
Traceback (most recent call last):
  File "/usr/lib/spark-current/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/spark-current/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:39092)

- You are not allowd to create multiple `SparkContext`. You have to stop existing one first with

In [35]:
sc.stop()

- Or use a fault tolerant function 

In [11]:
sc = pyspark.SparkContext.getOrCreate()

## Parallelized collections 

- Created by calling SparkContext’s parallelize method on an existing iterable or collection in your driver program. 

- The elements of the collection are copied to form a distributed dataset that can be operated on in parallel.

In [57]:
data = [1, 2, 3, 4, 5]
distData = sc.parallelize(data)
distData

data2 = [11, 21, 31, 41, 51]
distData2 = sc.parallelize(data2)
distData2

ParallelCollectionRDD[12] at parallelize at PythonRDD.scala:195

## External Datasets with RDD

- PySpark can create distributed datasets from any storage source supported by Hadoop, including your local file system, HDFS, Cassandra, HBase, Amazon S3, etc. Spark supports text files, SequenceFiles, and any other Hadoop InputFormat.

- Text file RDDs can be created using SparkContext’s textFile method. This method takes an URI for the file (either a local path on the machine, or a hdfs://, s3a://, etc URI) and reads it as a collection of lines. 

In [58]:
licenseFile = sc.textFile("/opt/apps/ecm/service/spark/2.4.4/package/spark-2.4.4-bin-hadoop2.7/licenses/LICENSE-vis.txt")
licenseFile

/opt/apps/ecm/service/spark/2.4.4/package/spark-2.4.4-bin-hadoop2.7/licenses/LICENSE-vis.txt MapPartitionsRDD[14] at textFile at NativeMethodAccessorImpl.java:0

## External Datasets with RDD

- **If using a path on the local filesystem, the file must also be accessible at the same path on worker nodes**. Either copy the file to all workers or use a network-mounted shared file system.

- All of Spark’s file-based input methods, including textFile, **support running on directories**, **compressed files**, and **wildcards** as well. For example, you can use `textFile("/my/directory")`, `textFile("/my/directory/*.txt")`, and `textFile("/my/directory/*.gz")`.

- The textFile method also takes an optional second argument for controlling the number of partitions of the file. By default, Spark creates one partition for each block of the file (blocks being 128MB by default in HDFS), but you can also ask for a higher number of partitions by passing a larger value. Note that you cannot have fewer partitions than blocks.

- `SparkContext.wholeTextFiles` lets you read a directory containing multiple small text files, and returns each of them as `(filename, content)` pairs. This is in contrast with textFile, which would return one record per line in each file.

- `RDD.saveAsPickleFile` and `SparkContext.pickleFile` support saving an RDD in a simple format consisting of pickled Python objects. Batching is used on pickle serialization, with default batch size 10.


## RDD Operations

### Simple MapReduce

In [59]:
lineLengths = licenseFile.map(lambda s: len(s))
totalLength = lineLengths.reduce(lambda a, b: a + b)

In [40]:
lineLengths

PythonRDD[4] at RDD at PythonRDD.scala:53

In [41]:
totalLength

385

## RDD Persistence


- One of the most important capabilities in Spark is persisting (or caching) a dataset in memory across operations. 

- When you persist an RDD, **each node stores any partitions** of it that it computes **in memory** and reuses them in other actions on that dataset (or datasets derived from it). 

- This allows future actions to be much faster (often by more than 10x). Caching is a key tool for iterative algorithms and fast interactive use.

- You can mark an RDD to be persisted using the `persist()` or `cache()` methods on it. The first time it is computed in an action, it will be kept in memory on the nodes. Spark’s cache is fault-tolerant – if any partition of an RDD is lost, it will automatically be recomputed using the transformations that originally created it.

In [46]:
lineLengths.persist()

PythonRDD[4] at RDD at PythonRDD.scala:53

In [47]:
lineLengths.cache() # same as persist() but use default storage level

PythonRDD[4] at RDD at PythonRDD.scala:53

## RDD different storage level


- `MEMORY_ONLY`	Store RDD as deserialized Java objects in the JVM. If the RDD does not fit in memory, some partitions will not be cached and will be recomputed on the fly each time they're needed. This is the default level.

- `MEMORY_AND_DISK`	Store RDD as deserialized Java objects in the JVM. If the RDD does not fit in memory, store the partitions that don't fit on disk, and read them from there when they're needed.

- `MEMORY_ONLY_SER` (Java and Scala)	Store RDD as serialized Java objects (one byte array per partition). This is generally more space-efficient than deserialized objects, especially when using a fast serializer, but more CPU-intensive to read.

- `MEMORY_AND_DISK_SER` (Java and Scala)	Similar to MEMORY_ONLY_SER, but spill partitions that don't fit in memory to disk instead of recomputing them on the fly each time they're needed.

- `DISK_ONLY` Store the RDD partitions only on disk.

- `MEMORY_ONLY_2`, `MEMORY_AND_DISK_2`, etc.	Same as the levels above, but replicate each partition on two cluster nodes.

- `OFF_HEAP` (experimental)	Similar to MEMORY_ONLY_SER, but store the data in off-heap memory. This requires off-heap memory to be enabled.

## RDD Unpersist 

- Spark automatically monitors cache usage on each node and drops out old data partitions in a least-recently-used (LRU) fashion. 

- If you would like to manually remove an RDD instead of waiting for it to fall out of the cache, use the RDD.unpersist() method.

In [45]:
lineLengths.unpersist()

PythonRDD[4] at RDD at PythonRDD.scala:53

## Broadcast Variables

- Broadcast variables allow the programmer to keep a read-only variable cached on each machine rather than shipping a copy of it with tasks. 

- They can be used, for example, to give every node a copy of a large input dataset in an efficient manner. Spark also attempts to distribute broadcast variables using efficient broadcast algorithms to reduce communication cost.


In [48]:
broadcastVar = sc.broadcast([1, 2, 3])
broadcastVar

- Broadcast variables are created from a variable `v` by calling `SparkContext.broadcast(v)`. 

- The broadcast variable is a wrapper around `v`, and its value can be accessed by calling the `.value` method.

- After the broadcast variable is created, it should be used instead of the value `v` in any functions run on the cluster so that **`v` is not shipped to the nodes more than once**. 

- In addition, the object `v` **should not be modified after it is broadcast** in order to ensure that all nodes get the same value of the broadcast variable (e.g. if the variable is shipped to a new node later).

In [49]:
broadcastVar.value

[1, 2, 3]

## Accumulators

- Accumulators are variables that are only “added” to through an associative and commutative operation and can therefore be efficiently supported in parallel. 

- They can be used to implement counters (as in MapReduce) or sums. Spark natively supports accumulators of numeric types, and programmers can add support for new types.

In [50]:
accum = sc.accumulator(0)
sc.parallelize([1, 2, 3, 4]).foreach(lambda x: accum.add(x))
accum.value

10

## Spark Lazy Evaluation

- “LAZY” the word itself indicates its meaning ‘not at the same time’. That means, it **evaluates something only when we require it**. 

- In accordance with a spark, it does not execute each operation right away, that means it does not start until we trigger any action. 

- Once an action is called all the transformations will execute in one go.

- It figured out that all transformations can be combined together into a single transformation and executed together. 

-  Benifits

    - It results in saving the time as well as reducing the space complexity.
    - The system works more efficiently with fewer resources and it also decreases the number of queries.
    - Lazy Evaluation works as a key building block in operations of Spark RDD.


## Spark Data Structures

- Local vector

In [52]:
import numpy as np
import scipy.sparse as sps
from pyspark.mllib.linalg import Vectors

# Use a NumPy array as a dense vector.
dv1 = np.array([1.0, 0.0, 3.0])
# Use a Python list as a dense vector.
dv2 = [1.0, 0.0, 3.0]
# Create a SparseVector.
sv1 = Vectors.sparse(3, [0, 2], [1.0, 3.0])
# Use a single-column SciPy csc_matrix as a sparse vector.
sv2 = sps.csc_matrix((np.array([1.0, 3.0]),
                      np.array([0, 2]),
                      np.array([0, 2])), shape=(3, 1))

- Local matrix

In [51]:
from pyspark.mllib.linalg import Matrix, Matrices
# Create a dense matrix ((1.0, 2.0), (3.0, 4.0), (5.0, 6.0))
dm2 = Matrices.dense(3, 2, [1, 2, 3, 4, 5, 6])
print(dm2)
# Create a sparse matrix ((9.0, 0.0), (0.0, 8.0), (0.0, 6.0))
sm = Matrices.sparse(3, 2, [0, 1, 3], [0, 2, 1], [9, 6, 8])
print(sm)

DenseMatrix([[1., 4.],
             [2., 5.],
             [3., 6.]])
3 X 2 CSCMatrix
(0,0) 9.0
(2,1) 6.0
(1,1) 8.0


- Labeled Points

In [ ]:
from pyspark.mllib.linalg import SparseVector
from pyspark.mllib.regression import LabeledPoint

# Create a labeled point with a positive label and a dense feature vector.
pos = LabeledPoint(1.0, [1.0, 0.0, 3.0])

# Create a labeled point with a negative label and a sparse feature vector.
neg = LabeledPoint(0.0, SparseVector(3, [0, 2], [1.0, 3.0]))

- Sparse data

In [ ]:
from pyspark.mllib.util import MLUtils
examples = MLUtils.loadLibSVMFile(sc,
           "/opt/apps/ecm/service/spark/2.4.4/package/spark-2.4.4-bin-hadoop2.7/data/mllib/sample_libsvm_data.txt")
print(examples)

### Distributed matrix

- RowMatrix

In [ ]:
from pyspark.mllib.linalg.distributed import RowMatrix
# Create an RDD of vectors.
rows = sc.parallelize([[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]])
# Create a RowMatrix from an RDD of vectors.
mat = RowMatrix(rows)
# Get its size.
m = mat.numRows()  # 4
n = mat.numCols()  # 3
# Get the rows as an RDD of vectors again.
rowsRDD = mat.rows
print(m,n,rowsRDD)

- IndexedRowMatrix

In [53]:
from pyspark.mllib.linalg.distributed import IndexedRow, IndexedRowMatrix
# Create an RDD of indexed rows.
#   - This can be done explicitly with the IndexedRow class:
indexedRows = sc.parallelize([IndexedRow(0, [1, 2, 3]),
                              IndexedRow(1, [4, 5, 6]),
                              IndexedRow(2, [7, 8, 9]),
                              IndexedRow(3, [10, 11, 12])])
#   - or by using (long, vector) tuples:
indexedRows = sc.parallelize([(0, [1, 2, 3]), (1, [4, 5, 6]),
                              (2, [7, 8, 9]), (3, [10, 11, 12])])

- BlockMatrix 

A BlockMatrix is a distributed matrix backed by an RDD of MatrixBlocks, where a MatrixBlock is a tuple of ((Int, Int), Matrix), where the (Int, Int) is the index of the block, and Matrix is the sub-matrix at the given index with size rowsPerBlock x colsPerBlock. BlockMatrix supports methods such as add and multiply with another BlockMatrix. BlockMatrix also has a helper function validate which can be used to check whether the BlockMatrix is set up properly.



In [ ]:
from pyspark.mllib.linalg import Matrices
from pyspark.mllib.linalg.distributed import BlockMatrix

# Create an RDD of sub-matrix blocks.
blocks = sc.parallelize([((0, 0), Matrices.dense(3, 2, [1, 2, 3, 4, 5, 6])),
                         ((1, 0), Matrices.dense(3, 2, [7, 8, 9, 10, 11, 12]))])

# Create a BlockMatrix from an RDD of sub-matrix blocks.
mat = BlockMatrix(blocks, 3, 2)

# Get its size.
m = mat.numRows()  # 6
n = mat.numCols()  # 2

# Get the blocks as an RDD of sub-matrix blocks.
blocksRDD = mat.blocks

# Convert to a LocalMatrix.
localMat = mat.toLocalMatrix()

# Convert to an IndexedRowMatrix.
indexedRowMat = mat.toIndexedRowMatrix()

# Convert to a CoordinateMatrix.
coordinateMat = mat.toCoordinateMatrix()

## Lab

- Run a simple Spark application with local mode。

- Understand the [basic data structure with RDD interface](https://spark.apache.org/docs/latest/mllib-data-types.html)。

- External Read

    - https://spark.apache.org/docs/latest/api/python/index.html
    - https://spark.apache.org/examples.html